In [1]:
import requests
import json

In [2]:
def search_wikipedia(query, lang='en'):
    URL = f"https://{lang}.wikipedia.org/w/api.php"
    PARAMS = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": query,
        "srlimit": 1
    }
    response = requests.get(URL, params=PARAMS)
    data = response.json()
    search_results = data.get('query', {}).get('search', [])
    if search_results:
        page_title = search_results[0]['title']
        return get_wikipedia_page(page_title, lang)
    return None

In [3]:
def get_wikipedia_page(title, lang='en'):
    URL = f"https://{lang}.wikipedia.org/w/api.php"
    PARAMS = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts|links",
        "exintro": True,
        "explaintext": True,
        "pllimit": "max"
    }
    response = requests.get(URL, params=PARAMS)
    data = response.json()
    pages = data.get('query', {}).get('pages', {})
    for page_id, page in pages.items():
        content = {
            "title": page.get('title'),
            "extract": page.get('extract'),
            "links": [link['title'] for link in page.get('links', [])]
        }
        return content
    return None

In [4]:
def search_wikidata(query):
    URL = "https://www.wikidata.org/w/api.php"
    PARAMS = {
        "action": "wbsearchentities",
        "format": "json",
        "type": "item",
        "search": query,
        "limit": 1
    }
    response = requests.get(URL, params=PARAMS)
    data = response.json()
    search_results = data.get('search', [])
    if search_results:
        entity_id = search_results[0]['id']
        return get_wikidata_entity(entity_id)
    return None

In [5]:
def get_wikidata_entity(entity_id):
    URL = "https://www.wikidata.org/w/api.php"
    PARAMS = {
        "action": "wbgetentities",
        "format": "json",
        "ids": entity_id,
        "props": "labels|descriptions|claims"
    }
    response = requests.get(URL, params=PARAMS)
    data = response.json()
    entities = data.get('entities', {})
    entity = entities.get(entity_id, {})
    content = {
        "id": entity_id,
        "labels": entity.get('labels', {}).get('en', {}).get('value'),
        "descriptions": entity.get('descriptions', {}).get('en', {}).get('value'),
        "claims": list(entity.get('claims', {}).keys())
    }
    return content

In [6]:
def search_wiktionary(query, lang='en'):
    URL = f"https://{lang}.wiktionary.org/w/api.php"
    PARAMS = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": query,
        "srlimit": 1
    }
    response = requests.get(URL, params=PARAMS)
    data = response.json()
    search_results = data.get('query', {}).get('search', [])
    if search_results:
        page_title = search_results[0]['title']
        return get_wiktionary_page(page_title, lang)
    return None

In [7]:
def get_wiktionary_page(title, lang='en'):
    URL = f"https://{lang}.wiktionary.org/w/api.php"
    PARAMS = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts|links",
        "exintro": True,
        "explaintext": True,
        "pllimit": "max"
    }
    response = requests.get(URL, params=PARAMS)
    data = response.json()
    pages = data.get('query', {}).get('pages', {})
    for page_id, page in pages.items():
        content = {
            "title": page.get('title'),
            "extract": page.get('extract'),
            "links": [link['title'] for link in page.get('links', [])]
        }
        return content
    return None

In [8]:
def search_commons(query):
    URL = "https://commons.wikimedia.org/w/api.php"
    PARAMS = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": query,
        "srlimit": 1
    }
    response = requests.get(URL, params=PARAMS)
    data = response.json()
    search_results = data.get('query', {}).get('search', [])
    if search_results:
        page_title = search_results[0]['title']
        return get_commons_page(page_title)
    return None

In [9]:
def get_commons_page(title):
    URL = "https://commons.wikimedia.org/w/api.php"
    PARAMS = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "imageinfo|links",
        "iiprop": "url",
        "pllimit": "max"
    }
    response = requests.get(URL, params=PARAMS)
    data = response.json()
    pages = data.get('query', {}).get('pages', {})
    for page_id, page in pages.items():
        imageinfo = page.get('imageinfo', [])
        image_url = imageinfo[0]['url'] if imageinfo else None
        content = {
            "title": page.get('title'),
            "image_url": image_url,
            "links": [link['title'] for link in page.get('links', [])]
        }
        return content
    return None

In [10]:
def collect_wikimedia_data(query):
    result = {
        "query": query,
        "wikipedia": search_wikipedia(query),
        "wikidata": search_wikidata(query),
        "wiktionary": search_wiktionary(query),
        "commons": search_commons(query)
    }
    return result

In [11]:
from bs4 import BeautifulSoup
from datetime import date
import os

In [12]:
def get_picture_of_a_day():
    url = 'https://ru.wikipedia.org/wiki/Заглавная_страница'
    try:
        response = requests.get(url)
        response.raise_for_status()  # Проверка успешности запроса
    except requests.RequestException as e:
        print(f"Ошибка при запросе страницы: {e}")
        return
    soup = BeautifulSoup(response.text, 'html.parser')
    potd_el = soup.find('div', id='main-potd')
    if not potd_el:
        print('Не удалось найти раздел "Изображение дня".')
        return
    img_el = potd_el.find('img')
    if not img_el:
        print('Не удалось найти изображение в разделе "Изображение дня".')
        return
    description = img_el.get('alt', 'Нет описания')
    srcset = img_el.get('srcset', '')
    if srcset:
        img_url = srcset.split(',')[-1].strip().split(' ')[0]
        if img_url.startswith('//'):
            img_link = 'https:' + img_url
        elif img_url.startswith('http'):
            img_link = img_url
        else:
            img_link = 'https://' + img_url
    else:
        img_src = img_el.get('src', '')
        img_link = 'https:' + img_src if img_src.startswith('//') else img_src
    try:
        img_response = requests.get(img_link)
        img_response.raise_for_status()
    except requests.RequestException as e:
        print(f"Ошибка при загрузке изображения: {e}")
        return
    try:
        with open('Картинка дня.jpg', 'wb') as img_file:
            img_file.write(img_response.content)
        with open('Описание.txt', 'w', encoding='utf-8') as desc_file:
            desc_file.write(description)
        print('Изображение дня и его описание успешно сохранены.')
    except IOError as e:
        print(f"Ошибка при сохранении файлов: {e}")

In [13]:
def main():
    query = input("Ваш запрос: ")
    data = collect_wikimedia_data(query)
    print(json.dumps(data, indent=4, ensure_ascii=False))

if __name__ == "__main__":
    main()
    get_picture_of_a_day()

Ваш запрос:  Москва


{
    "query": "Москва",
    "wikipedia": {
        "title": "Moscow",
        "extract": "Moscow is the capital and largest city of Russia. The city stands on the Moskva River in Central Russia, with a population estimated at over 13 million residents within the city limits, over 19.1 million residents in the urban area, and over 21.5 million residents in its metropolitan area. The city covers an area of 2,511 square kilometers (970 sq mi), while the urban area covers 5,891 square kilometers (2,275 sq mi), and the metropolitan area covers over 26,000 square kilometers (10,000 sq mi). Moscow is among the world's largest cities, being the most populous city in its entirety in Europe, the largest urban and metropolitan area in Europe, and the largest city by land area on the European continent.\nFirst documented in 1147, Moscow grew to serve as the capital of the Grand Duchy of Moscow. When the Tsardom of Russia was proclaimed, Moscow remained the political and economic center for most o